# **PRO - Consolidado por semana**

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_col = pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Colombia.csv")
data_atl = pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Atlantico.csv")
data_bqui= pd.read_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Barranquilla.csv")

In [3]:

def resumen_por_semana(df):
    # columnas a ignorar
    ignorar = ['edad_años', 'periodo', 'fecha_inicio_sintomas', 'fecha_consulta',
               'fecha_hospitalizacion', 'fecha_notificacion', 'fecha_nacimiento', 'año', 'semana',
               'direccion', 'localidad', 'barrio', 'periodo']
    
    # seleccionar solo columnas categóricas relevantes
    cols_cat = [c for c in df.columns if c not in ignorar]
    
    # combinar año y semana en una sola columna
    df['año_semana'] = df['año'].astype(str) + '_' + df['semana'].astype(str)
    
    resultados = []
    
    # para cada columna categórica (excepto año y semana)
    for col in cols_cat:
        if col not in ['año', 'semana']:
            conteo = df.groupby(['año_semana', col]).size().unstack(fill_value=0)
            resultados.append(conteo)
    
    # unir todos los conteos
    resumen = pd.concat(resultados, axis=1)
    
    # resetear el índice
    resumen = resumen.reset_index().fillna(0)
    resumen[['año', 'semana']] = resumen['año_semana'].str.split('_', expand=True).astype(int)
    resumen = resumen.sort_values(by=['año', 'semana']).reset_index(drop=True)
    resumen['año_semana'] = resumen['año'].astype(str) + '_' + resumen['semana'].astype(str)

    total_eventos = df.groupby('año_semana')['evento'].count().reset_index(name='total_casos')
    resumen = resumen.merge(total_eventos, on='año_semana', how='left')

    return resumen


In [ ]:
consolidado_atl = resumen_por_semana(data_atl)
consolidado_col = resumen_por_semana(data_col)
consolidado_bquilla = resumen_por_semana(data_bqui)

,año_semana,Femenino,Masculino,Adolescencia,Adulto,Adulto mayor,Ancianidad,Infancia,Menor de 1 año,Primera infancia,...,SABANALARGA,SANTA LUCÍA,SANTO TOMÁS,SOLEDAD,SUAN,TUBARÁ,USIACURÍ,año,semana,total_casos
0,2017_1,1,1,0,2,0,0,0,0,0,...,0,0,0,1,0,0,0,2017,1,2
1,2017_2,0,3,1,1,0,0,1,0,0,...,0,0,1,2,0,0,0,2017,2,3
2,2017_3,0,2,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,2017,3,2
3,2017_4,2,1,0,1,0,0,1,0,1,...,0,0,0,2,0,0,0,2017,4,3
4,2017_5,1,3,1,0,1,0,1,1,0,...,0,0,0,2,0,0,0,2017,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2023_48,55,65,40,48,7,0,18,0,7,...,1,0,1,54,1,1,1,2023,48,120
360,2023_49,74,61,57,39,10,0,17,0,12,...,6,0,4,68,2,1,1,2023,49,135
361,2023_50,56,62,47,43,6,1,13,1,7,...,4,0,2,67,2,0,1,2023,50,118
362,2023_51,50,60,48,35,4,0,12,0,11,...,3,0,3,54,0,1,1,2023,51,110


In [7]:
consolidado_bquilla.to_csv("../DATA/Resumen/consolidado_barranquilla.csv", index=False)
consolidado_atl.to_csv("../DATA/Resumen/consolidado_atlantico.csv", index=False)
consolidado_col.to_csv("../DATA/Resumen/consolidado_colombia.csv", index=False)